In [1]:
import os
from dotenv import load_dotenv
import numpy as np
import tensorflow as tf
import pandas as pd
import datetime

load_dotenv(override=True)

DATA_PATH = os.getenv('DATA_PATH')
print(DATA_PATH)

# Choose subontology (CCO, MFO or BPO)
SO = 'BPO'

2023-05-16 17:58:32.855877: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-16 17:58:33.332120: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


/mnt/e/ML/cafa-5-protein-function-prediction


## Reading fasta, obo and tsv files

In [2]:
from Bio import SeqIO

sequences = [rec.seq for rec in SeqIO.parse(os.path.join(DATA_PATH, "Train/train_sequences.fasta"),"fasta")]
ids = [rec.id for rec in SeqIO.parse(os.path.join(DATA_PATH, "Train/train_sequences.fasta"),"fasta")]

In [3]:
print("There are {} sequences in the dataset.".format(len(sequences)))

There are 142246 sequences in the dataset.


In [4]:
import networkx
import obonet

# Read the taxrank ontology
url = os.path.join(DATA_PATH, "Train/go-basic.obo")
graph = obonet.read_obo(url)



In [5]:
df = pd.read_csv(os.path.join(DATA_PATH, "Train/train_terms.tsv"), sep='\t')
dfSO = df.loc[df["aspect"]==SO]
uniqueTerms = dfSO["term"].unique()
termsArr = list(dfSO["term"].to_numpy())

uniqueTermsDict={}
for i,el in enumerate(uniqueTerms):
    uniqueTermsDict[el] = i
    
print(dfSO.shape)
df=dfSO

(3497732, 3)


Test for the first entry:

In [6]:
df.set_index("EntryID", inplace=True)
df.head(10)

,term,aspect
EntryID,,
A0A009IHW8,GO:0008152,BPO
A0A009IHW8,GO:0034655,BPO
A0A009IHW8,GO:0072523,BPO
A0A009IHW8,GO:0044270,BPO
A0A009IHW8,GO:0006753,BPO
A0A009IHW8,GO:1901292,BPO
A0A009IHW8,GO:0044237,BPO
A0A009IHW8,GO:1901360,BPO
A0A009IHW8,GO:0008150,BPO


In [7]:
%timeit df.loc["A0A021WW32"]

20 µs ± 306 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [8]:
testID = df.index.to_list()[0]

In [9]:

df.loc[testID]

,term,aspect
EntryID,,
A0A009IHW8,GO:0008152,BPO
A0A009IHW8,GO:0034655,BPO
A0A009IHW8,GO:0072523,BPO
A0A009IHW8,GO:0044270,BPO
A0A009IHW8,GO:0006753,BPO
A0A009IHW8,GO:1901292,BPO
A0A009IHW8,GO:0044237,BPO
A0A009IHW8,GO:1901360,BPO
A0A009IHW8,GO:0008150,BPO


In [10]:
np.unique(df["aspect"].to_numpy())

array(['BPO'], dtype=object)

## GO analysis

In [11]:
item_counts = df["term"].value_counts()
print(item_counts[0:20])

GO:0008150    92210
GO:0009987    61293
GO:0065007    41457
GO:0050789    39256
GO:0050794    33888
GO:0050896    31098
GO:0008152    30448
GO:0032501    29274
GO:0032502    28680
GO:0071704    28274
GO:0048856    27366
GO:0044237    25273
GO:0044238    24419
GO:0007275    21784
GO:0006807    21579
GO:0071840    19458
GO:0019222    19174
GO:0016043    18486
GO:0048518    18364
GO:0043170    17450
Name: term, dtype: int64


In [12]:
id_to_name = {id_: data.get('name') for id_, data in graph.nodes(data=True)}
name_to_id = {data['name']: id_ for id_, data in graph.nodes(data=True) if 'name' in data}
print(id_to_name['GO:0031981'] )

nuclear lumen


In [13]:
print(id_to_name['GO:0070013'] )
print(networkx.ancestors(graph, 'GO:0070013'))
print(networkx.descendants(graph, 'GO:0070013'))

paths = networkx.all_simple_paths(
    graph,
    source='GO:0070013',
    target=name_to_id['molecular_function']
)

for path in paths:
    print('•', ' ⟶ '.join(id_to_name[node] for node in path))

intracellular organelle lumen
{'GO:0070449', 'GO:0005763', 'GO:0036224', 'GO:0060202', 'GO:0099086', 'GO:0000794', 'GO:0031261', 'GO:0033276', 'GO:0005668', 'GO:1990934', 'GO:0070082', 'GO:0000330', 'GO:0061201', 'GO:0030896', 'GO:0097424', 'GO:0035363', 'GO:0043596', 'GO:0033985', 'GO:0031089', 'GO:0005762', 'GO:0016606', 'GO:0036014', 'GO:0031907', 'GO:0045244', 'GO:1904724', 'GO:0070860', 'GO:0070692', 'GO:0034493', 'GO:0034774', 'GO:0034388', 'GO:0106172', 'GO:0014803', 'GO:0005759', 'GO:1990826', 'GO:0015030', 'GO:0043494', 'GO:0005662', 'GO:0034492', 'GO:0043599', 'GO:0060205', 'GO:0000817', 'GO:0032021', 'GO:0008023', 'GO:0016607', 'GO:0071601', 'GO:0071339', 'GO:0017087', 'GO:0031905', 'GO:0106098', 'GO:0008024', 'GO:0099020', 'GO:0005714', 'GO:0070823', 'GO:0062113', 'GO:0043601', 'GO:0031906', 'GO:0106176', 'GO:0016605', 'GO:0016593', 'GO:0005775', 'GO:0031972', 'GO:0031096', 'GO:0098899', 'GO:0005665', 'GO:0000120', 'GO:0005669', 'GO:0009841', 'GO:0106173', 'GO:0097486', 'GO

## Label encoding

The task is a multilabel classification: The output has several possible targets (Gene Ontologies) but each can only be 1 (existing) or 0 (non existing)

In [14]:
from sklearn.preprocessing import MultiLabelBinarizer

topGOs= item_counts
topGOs=topGOs.index.to_list()

mlb = MultiLabelBinarizer()
mlb.fit([topGOs])

dftest=df.loc[testID]
indices = dftest["term"].to_numpy()
print(indices)
print(mlb.transform([indices]))
print(len(mlb.classes_))

['GO:0008152' 'GO:0034655' 'GO:0072523' 'GO:0044270' 'GO:0006753'
 'GO:1901292' 'GO:0044237' 'GO:1901360' 'GO:0008150' 'GO:1901564'
 'GO:1901565' 'GO:0009117' 'GO:0006139' 'GO:0044281' 'GO:0046496'
 'GO:0019362' 'GO:0046483' 'GO:0055086' 'GO:0044248' 'GO:0019439'
 'GO:0019637' 'GO:0006807' 'GO:0019677' 'GO:1901361' 'GO:0006163'
 'GO:0046700' 'GO:0009987' 'GO:0006725' 'GO:0006796' 'GO:0034641'
 'GO:0072521' 'GO:0071704' 'GO:0019364' 'GO:1901575' 'GO:0072526'
 'GO:0046434' 'GO:0009166' 'GO:0072524' 'GO:0006195' 'GO:0009056'
 'GO:0044238' 'GO:0006793' 'GO:0019674']
[[0 0 0 ... 0 0 0]]
21285


## Amino acids encoding

In [15]:
aminos_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'X']

In [16]:
aa_dict = {'A': 1, 'B':24, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10, 'M': 11, 'N': 12, 'O': 21, 'P': 13, 'Q': 14, 'R': 15, 'S': 16, 'T': 17, 'U': 22, 'V': 18, 'W': 19, 'Y': 20, 'X':30, 'Z':23}

## Build Dataset

In [17]:
seqLengths = [len(seq) for seq in sequences]
maxLen = max(seqLengths)
print("The max. length of the sequences is {}".format(maxLen))

The max. length of the sequences is 35375


In [18]:

dfAll=pd.read_csv(os.path.join(DATA_PATH, "Train/train_terms.tsv"), sep='\t')
# dfAll.set_index("EntryID", inplace=True)
soEntries = dfAll.loc[dfAll["aspect"]==SO]
soEntryIds = soEntries["EntryID"].unique()

# print(len(seqEntries))
print(soEntryIds)

SoSequences = []
for entry in soEntryIds:
    SoSequences.append(sequences[ids.index(entry)])

print(len(SoSequences))

['A0A009IHW8' 'A0A021WW32' 'A0A023FFD0' ... 'X5L1L5' 'X5L565' 'X5M5N0']
92210


In [19]:
import warnings

TRAIN_VAL_SPLIT = 0.7

#Use numpy vectorize to speed up the mapping (hopefully)
mapping = lambda x: aa_dict[x]
vectMapping = np.vectorize(mapping)

# Shuffle the data
import random
random.seed(516213)
c = list(zip(SoSequences, soEntryIds))
random.shuffle(c)
sequencesShuffle, idsShuffle = zip(*c)

#reduce data for now
# sequencesShuffle = sequencesShuffle[0:10000]
# idsShuffle = idsShuffle[0:10000]

#Train Validation Split
split = int(np.floor(len(sequencesShuffle)*TRAIN_VAL_SPLIT))
print(split)
trainSeq = sequencesShuffle[0:split]
valSeq = sequencesShuffle[split+1:]
trainIds = idsShuffle[0:split]
valIds = idsShuffle[split+1:]


def generator(padding=True):
    for i,seq in enumerate(trainSeq):
        entryId = trainIds[i]
        labelData = df.loc[entryId]
        
        # indices = labelData["termToken"].to_numpy()
        indices = labelData["term"].to_numpy()

        with warnings.catch_warnings():
            #supress the warnings for unknown classes
            warnings.simplefilter("ignore")
            y = mlb.transform([indices])
        
        arr = np.array(seq)
        mappedArr = vectMapping(arr)
        if padding:
            padWidth = maxLen - arr.size
            mappedArr = np.pad(mappedArr, (0, padWidth))
        yield mappedArr,y[0]

def generatorVal(padding=True):
    for i,seq in enumerate(valSeq):
        entryId = valIds[i]
        labelData = df.loc[entryId]
        
        # indices = labelData["termToken"].to_numpy()
        indices = labelData["term"].to_numpy()

        with warnings.catch_warnings():
            #supress the warnings for unknown classes
            warnings.simplefilter("ignore")
            y = mlb.transform([indices])
        
        arr = np.array(seq)
        mappedArr = vectMapping(arr)
        if padding:
            padWidth = maxLen - arr.size
            mappedArr = np.pad(mappedArr, (0, padWidth))
        yield mappedArr,y[0]
        

64546


In [20]:
g = generator()
test = next(g)
print("The first sample sequence: {}".format(test[0]))
print("The first sample has {} classes".format(np.count_nonzero(test[1])))

The first sample sequence: [11 15  9 ...  0  0  0]
The first sample has 127 classes


## Tensorflow Classification

In [21]:
import tensorflow as tf


dataset = tf.data.Dataset.from_generator(generator, output_signature=(
         tf.TensorSpec(shape=(maxLen,), dtype=tf.int32),
         tf.TensorSpec(shape=(len(mlb.classes_),), dtype=tf.int32)))
print(list(dataset.take(1)))

datasetVal = tf.data.Dataset.from_generator(generatorVal, output_signature=(
         tf.TensorSpec(shape=(None,), dtype=tf.int32),
         tf.TensorSpec(shape=(len(mlb.classes_),), dtype=tf.int32)))

2023-05-16 17:59:38.362199: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-16 17:59:38.385464: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-16 17:59:38.385793: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-16 17:59:38.387172: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-16 17:59:38.387470: I tensorflow/compile

[(<tf.Tensor: shape=(35375,), dtype=int32, numpy=array([11, 15,  9, ...,  0,  0,  0], dtype=int32)>, <tf.Tensor: shape=(21285,), dtype=int32, numpy=array([0, 0, 0, ..., 0, 0, 0], dtype=int32)>)]


In [22]:
%load_ext tensorboard


In [23]:
from tensorflow.keras import layers

VOCAB_SIZE=len(aa_dict)
EMBED_DIM=100

def createModel():
    inputs = tf.keras.Input(shape=(maxLen,))
    x=layers.Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True, name="embedding")(inputs)
    x=layers.GlobalAveragePooling1D()(x)
    x=layers.Reshape((EMBED_DIM,1))(x)
    x=layers.Conv1D(8, 7)(x)
    x=layers.Conv1D(8, 7)(x)
    x=layers.Conv1D(8, 7)(x)
    x=layers.Conv1D(16, 7)(x)
    x=layers.Conv1D(16, 7)(x)
    x=layers.Conv1D(16, 7, strides=2)(x)
    x=layers.Conv1D(16, 7, strides=2)(x)
    # x=layers.Conv1D(32, 5, activation=tf.keras.activations.relu)(x)
    # x=layers.Conv1D(32, 5, activation=tf.keras.activations.relu)(x)
    # x=layers.Conv1D(32, 5, activation=tf.keras.activations.relu)(x)
    x=layers.Flatten()(x)
    x=layers.Dense(32)(x)
    x=layers.LeakyReLU()(x)
    x=layers.Dense(32)(x)
    x=layers.LeakyReLU()(x)
    outputs=layers.Dense(len(mlb.classes_), activation=tf.keras.activations.sigmoid)(x)
    # outputs=layers.Softmax()(x)

    return tf.keras.Model(inputs=inputs, outputs=outputs, name="embedConvModel")

model = createModel()

model.summary()


Model: "embedConvModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 35375)]           0         
                                                                 
 embedding (Embedding)       (None, 35375, 100)        2500      
                                                                 
 global_average_pooling1d (G  (None, 100)              0         
 lobalAveragePooling1D)                                          
                                                                 
 reshape (Reshape)           (None, 100, 1)            0         
                                                                 
 conv1d (Conv1D)             (None, 94, 8)             64        
                                                                 
 conv1d_1 (Conv1D)           (None, 88, 8)             456       
                                                    

In [24]:

VOCAB_SIZE=len(aa_dict)
EMBED_DIM=200

def createRnnModel():
    inputs = tf.keras.Input(shape=(maxLen,))
    x = tf.keras.layers.Masking(0)(inputs)
    x=layers.Embedding(VOCAB_SIZE, EMBED_DIM, name="embedding")(x)

    # x = layers.Bidirectional(layers.LSTM(32, return_sequences=True))(x)
    # x = layers.Bidirectional(layers.GRU(16, return_sequences=True))(x)
    x = layers.Bidirectional(layers.GRU(16))(x)
    # x = layers.LSTM(32)(x)
    x = layers.Dense(16)(x)
    x = layers.LeakyReLU()(x)
    outputs=layers.Dense(len(mlb.classes_), activation=tf.keras.activations.sigmoid)(x)
    # outputs=layers.Softmax()(x)

    return tf.keras.Model(inputs=inputs, outputs=outputs, name="embedRnnModel")

# model = createRnnModel()

# model.summary()

In [25]:
import matplotlib.pyplot as plt
#Learning rate schedule
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10,
    decay_rate=0.9,
    staircase=False)
# step = np.linspace(0,1000)
# lr = lr_schedule(step)
# plt.figure(figsize = (8,6))
# plt.yscale("log")
# plt.plot(step, lr)
# plt.ylim([0,max(plt.ylim())])
# plt.xlabel('step')
# _ = plt.ylabel('Learning Rate')

In [34]:
BATCH_SIZE=32
LOG_INTERVAL=10
epochs = 20
saveModel=False


print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

log_dir = "./logs/"+model.name+"/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_"+SO
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,
                                                      write_graph=True, update_freq=5)


summary_writer = tf.summary.create_file_writer(log_dir)

# Instantiate an optimizer .
# optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
# optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
optimizer = tf.keras.optimizers.AdamW(learning_rate=0.01)

# Instantiate a loss function.
loss_fn = tf.keras.losses.BinaryCrossentropy()

train_acc_metric = tf.keras.metrics.BinaryAccuracy()
val_acc_metric = tf.keras.metrics.BinaryAccuracy()

batchedDataset = dataset.batch(BATCH_SIZE, drop_remainder=False)
batchedDatasetVal = datasetVal.batch(BATCH_SIZE, drop_remainder=False)
# print(batchedDataset.take(1))

batchedDataset = batchedDataset.cache(os.path.join(DATA_PATH, "datasetCache"+SO))
batchedDatasetVal = batchedDatasetVal.cache(os.path.join(DATA_PATH, "datasetCacheVal"+SO))

# @tf.function()
def trainStep(x_batch_train, y_batch_train):
    # Open a GradientTape to record the operations run
    # during the forward pass, which enables auto-differentiation.
    with tf.GradientTape() as tape:

        # Run the forward pass of the layer.
        # The operations that the layer applies
        # to its inputs are going to be recorded
        # on the GradientTape.
        probs = model(x_batch_train, training=True) 

        # Compute the loss value for this minibatch.
        loss_value = loss_fn(y_batch_train, probs)

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss.
    grads = tape.gradient(loss_value, model.trainable_weights)
    #Gradient clipping
    # grads = [tf.clip_by_norm(g, 2.0) for g in grads]

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    train_acc_metric.update_state(y_batch_train, probs)
    return loss_value, grads

maxStep=0

for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch+1,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(batchedDataset):

        loss_value, grads=trainStep(x_batch_train,y_batch_train)

        # Log 
        if step % LOG_INTERVAL == 0:
            template = 'Epoch {}/Step {}, Loss: {:.5f}, Accuracy: {:.5f}, lr: {:.5f}'
            print (template.format(epoch+1, step,loss_value, 
                                    train_acc_metric.result(), optimizer.learning_rate.numpy()))
            # print([tf.norm(grad, ord=2).numpy() for grad in grads])
            with summary_writer.as_default():
                tf.summary.scalar('loss', loss_value, step=maxStep*epoch+step)
                tf.summary.scalar('accuracy', train_acc_metric.result(), step=maxStep*epoch+step)
                summary_writer.flush()

    
    train_acc_metric.reset_states()

    maxStep=step

    print("Epoch finished. Start validation")
    for x_batch_val, y_batch_val in batchedDatasetVal:
        valProbs = model(x_batch_val, training=False)
        # Update val metrics
        val_acc_metric.update_state(y_batch_val, valProbs)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    with summary_writer.as_default():
        tf.summary.scalar('valAcc', float(val_acc), step=epoch)
        summary_writer.flush()
    if saveModel:
      model.save(os.path.join(DATA_PATH, "model_"+SO+"_epoch_{}_valAcc{:.3f}".format(epoch, float(val_acc))))

Num GPUs Available:  1

Start of epoch 1


2023-05-16 18:07:17.970012: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1/Step 0, Loss: 0.00810, Accuracy: 0.99816, lr: 0.01000
Epoch 1/Step 10, Loss: 0.09193, Accuracy: 0.91396, lr: 0.01000
Epoch 1/Step 20, Loss: 0.07632, Accuracy: 0.95323, lr: 0.01000
Epoch 1/Step 30, Loss: 0.30883, Accuracy: 0.96625, lr: 0.01000
Epoch 1/Step 40, Loss: 0.38556, Accuracy: 0.94951, lr: 0.01000
Epoch 1/Step 50, Loss: 0.98073, Accuracy: 0.95879, lr: 0.01000
Epoch 1/Step 60, Loss: 0.12851, Accuracy: 0.96503, lr: 0.01000
Epoch 1/Step 70, Loss: 1.07380, Accuracy: 0.96840, lr: 0.01000
Epoch 1/Step 80, Loss: 0.60971, Accuracy: 0.97191, lr: 0.01000
Epoch 1/Step 90, Loss: 0.24330, Accuracy: 0.97464, lr: 0.01000
Epoch 1/Step 100, Loss: 0.09811, Accuracy: 0.97680, lr: 0.01000
Epoch 1/Step 110, Loss: 0.22531, Accuracy: 0.97851, lr: 0.01000
Epoch 1/Step 120, Loss: 0.08843, Accuracy: 0.98007, lr: 0.01000
Epoch 1/Step 130, Loss: 0.10498, Accuracy: 0.98131, lr: 0.01000
Epoch 1/Step 140, Loss: 0.12258, Accuracy: 0.98241, lr: 0.01000
Epoch 1/Step 150, Loss: 0.03428, Accuracy: 0.98336,

KeyboardInterrupt: 

In [ ]:
# for layer in model.layers: print(layer.get_config(), layer.get_weights())

In [32]:

probs= model.predict(tf.expand_dims(list(datasetVal.take(32))[0][0], 0))
prediction= [1 if p > 0.5 else 0 for p in probs[0]]
# classes = np.argwhere(prediction)
mlb.inverse_transform(np.array([prediction]))


1/1 [==============================] - 0s 13ms/step


2023-05-16 18:04:31.675169: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


[('GO:0008150', 'GO:0009987', 'GO:0050789', 'GO:0050794', 'GO:0065007')]